In [1]:
#surrogate models
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split

data_file = './body_vectors_drags.csv'
df = TabularDataset(data_file)

train_df, test_df = train_test_split(df, test_size=0.2, random_state=777)

#exclue the first two columns of train data
train_data = train_df.drop(columns=['i', 'name'])
train_data.head()

/home/xli/anaconda3/envs/surrogate/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,...,dim_56,dim_57,dim_58,dim_59,dim_60,dim_61,dim_62,dim_63,dim_64,drag
61,-6.639910,-2.575970,-4.333867,0.348634,-6.591602,4.137893,3.448946,9.801783,-4.944950,-1.340411,...,4.211274,0.876610,2.535916,4.460216,7.625662,5.399125,2.165930,1.335564,-6.787674,0.375
354,5.044733,2.543681,-3.247805,-5.378260,0.549456,-1.075264,6.259791,5.443143,1.828774,5.774058,...,5.602760,-0.438914,-1.589318,4.962416,2.475589,-6.530942,-0.353212,1.797986,-2.464886,0.374
358,-0.939149,0.704528,0.749587,-1.643254,-3.556550,0.293420,3.039790,-3.084175,-3.810488,3.759489,...,-4.860520,-0.330673,2.862438,-0.245570,2.726983,-3.573323,-3.095024,0.883057,-0.973858,0.435
275,-1.177986,1.175732,-1.834498,1.306601,-0.493694,1.298445,2.197183,4.541283,1.859666,3.284920,...,-2.512394,-5.162504,0.902922,7.131485,-3.511894,4.393147,-5.885215,0.389388,5.135883,0.437
18,0.390329,-1.530953,-2.521950,-4.864147,-0.742480,0.146031,-0.156536,1.740131,-2.943921,5.280754,...,1.704322,5.339616,3.019771,6.336881,1.606373,-2.136702,-7.156015,3.795955,1.753440,0.367


In [2]:
label = 'drag'
print("Summary of class variable: \n", train_data[label].describe())

Summary of class variable: 
 count    351.000000
mean       0.398513
std        0.060013
min        0.278000
25%        0.353000
50%        0.394000
75%        0.435000
max        0.598000
Name: drag, dtype: float64


In [3]:
import os
save_path = './agModels-body'  # specifies folder to store trained models
if not os.path.exists(save_path):
    os.makedirs(save_path)

metric = 'root_mean_squared_error' #Regression:mean_absolute_error, mean_squared_error,root_mean_squared_error (default), r2
predictor = TabularPredictor(label=label, path=save_path, eval_metric=metric).fit(train_data, presets='best_quality', num_bag_folds=0, num_bag_sets=1)

Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=0, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "./agModels-body/"
AutoGluon Version:  0.7.0
Python Version:     3.10.10
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #51-Ubuntu SMP Mon Jul 4 06:41:22 UTC 2022
Train Data Rows:    351
Train Data Columns: 64
Label Column: drag
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (0.598, 0.278, 0.39851, 0.06001)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:       

In [4]:
test_data = test_df.drop(columns=['i', 'name'])
# val_data.head()
y_val = test_data[label]
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,...,dim_55,dim_56,dim_57,dim_58,dim_59,dim_60,dim_61,dim_62,dim_63,dim_64
46,3.262418,4.623484,5.207058,1.474604,5.149589,1.366774,7.350364,4.988407,-4.451728,2.715955,...,3.239450,0.271440,-1.450751,6.201051,2.322523,-0.651795,-2.503192,-1.329733,5.614099,4.784914
101,-1.815653,0.467063,-0.713573,-2.519432,-6.833046,3.925573,3.131740,2.785505,-0.169789,3.797550,...,8.718956,-4.833921,-1.387358,1.010315,0.467406,-7.262593,0.406025,-4.430325,-4.493312,3.210908
175,-0.128397,-0.347771,-4.491439,-0.143762,-1.139696,1.523660,4.283424,1.275272,-3.277825,-0.944104,...,2.371663,-0.860736,-2.739771,4.489244,4.020984,1.973119,3.365454,-2.144289,1.310637,-1.411514
9,-0.334880,-0.358754,-5.121314,-1.869275,0.053525,2.998930,-0.482418,1.283854,-1.355309,-0.988547,...,4.671710,-1.422821,0.833765,5.060172,3.785773,1.105779,2.158277,-2.020028,-0.746978,1.548296
136,0.339348,2.649365,-1.346519,0.872298,-2.672357,1.983254,2.075392,6.573480,-0.809723,-2.231186,...,0.290699,-0.446408,-5.342670,5.205126,5.651273,-2.507123,1.437673,-3.640510,-1.535356,4.507751


In [6]:
%%capture log_output
%config InlineBackend.figure_format = 'retina'
%config Application.log_level = 'DEBUG'
%config IPCompleter.greedy = True

predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file
y_pred = predictor.predict(test_data_nolab)
for item in y_pred:
    print(item)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_val, y_pred=y_pred, auxiliary_metrics=True)
print(perf)

results = predictor.fit_summary(show_plot=True)
print(results)
print(predictor.leaderboard(test_data, silent=True))

with open('./output_body.log', 'w') as f:
    f.write(log_output.stdout)

Evaluation: root_mean_squared_error on test data: -0.050034147274332794
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "root_mean_squared_error": -0.050034147274332794,
    "mean_squared_error": -0.002503415893469624,
    "mean_absolute_error": -0.03686644413525408,
    "r2": 0.1486749360125207,
    "pearsonr": 0.3991500258208202,
    "median_absolute_error": -0.02989608228206636
}


In [ ]:
print("AutoGluon infers problem type is: ", predictor.problem_type)
print("AutoGluon identified the following types of features:")
print(predictor.feature_metadata)

AutoGluon infers problem type is:  regression
AutoGluon identified the following types of features:
('float', []) : 64 | ['dim_1', 'dim_2', 'dim_3', 'dim_4', 'dim_5', ...]


In [7]:
train_data_pred = predictor.predict(train_data, model='WeightedEnsemble_L2')
for item in train_data_pred:
    print(item)

0.39179033041000366
0.3810622990131378
0.42376548051834106
0.4437100291252136
0.363418310880661
0.3789520263671875
0.403035044670105
0.4026564955711365
0.3625396490097046
0.3624393939971924
0.3658306300640106
0.4688659608364105
0.4446702003479004
0.3616076707839966
0.43355903029441833
0.4477322995662689
0.3285706639289856
0.4058804214000702
0.48094141483306885
0.3682215213775635
0.3887735605239868
0.45548808574676514
0.3750140070915222
0.44727540016174316
0.379258930683136
0.42059534788131714
0.3870013356208801
0.4131103754043579
0.3449678421020508
0.3451216518878937
0.41435036063194275
0.43246224522590637
0.46546685695648193
0.45046740770339966
0.35544735193252563
0.36518368124961853
0.3741125166416168
0.41998228430747986
0.4059818387031555
0.3419978618621826
0.48325586318969727
0.37301015853881836
0.3563459515571594
0.36194753646850586
0.373629629611969
0.4176792800426483
0.4082556962966919
0.4265376925468445
0.371904194355011
0.4069189429283142
0.38090550899505615
0.3891986012458801

In [8]:
test_data_pred = predictor.predict(test_data, model='WeightedEnsemble_L2')
for item in test_data_pred:
    print(item)

0.3680879473686218
0.3668398857116699
0.4162694215774536
0.3744370937347412
0.39999914169311523
0.41955405473709106
0.43098902702331543
0.44742828607559204
0.3989654779434204
0.3661521077156067
0.38885247707366943
0.4264290928840637
0.3580113649368286
0.41094690561294556
0.41759610176086426
0.4063419997692108
0.37049055099487305
0.3720453083515167
0.36963918805122375
0.36872708797454834
0.42646923661231995
0.39879709482192993
0.36635568737983704
0.4158444106578827
0.442308634519577
0.3707406520843506
0.42948588728904724
0.38404983282089233
0.37580224871635437
0.3908371329307556
0.4203270971775055
0.3802730441093445
0.3808373808860779
0.4379156231880188
0.4138909876346588
0.3898378610610962
0.37178879976272583
0.3968990445137024
0.3959178626537323
0.38958024978637695
0.43402737379074097
0.3811335563659668
0.4189939796924591
0.3743710517883301
0.3966352939605713
0.401253342628479
0.39957699179649353
0.37944507598876953
0.335768461227417
0.38161516189575195
0.40289002656936646
0.339003026